In [37]:
import pandas as pd
from io import StringIO
import requests
import logging
import json
from datetime import datetime
import bcrypt
from functools import wraps
import pytz


In [38]:
final_df = pd.read_csv('../data/course_members_output.csv')
username = 'admin'

In [43]:
def updateMetrics(final_df, username):
    # Convert time columns to datetime
    final_df['Completed Time'] = pd.to_datetime(final_df['Completed Time'])
    final_df['Enrollment Time'] = pd.to_datetime(final_df['Enrollment Time'])
    final_df['Last Login'] = pd.to_datetime(final_df['Last Login'])
    final_df['Enrolled date'] = pd.to_datetime(final_df['Enrolled date'])
    completion_percentage = final_df['Course progress'].mean()
    # Convert 'Enrolled date' to datetime
    final_df['Enrolled date'] = pd.to_datetime(final_df['Enrolled date'], errors='coerce')
    # Extract month and year from 'Enrolled date'
    final_df['Enrollment Month'] = final_df['Enrolled date'].dt.to_period('M')
    # Count enrollments per month and sort by the actual month (newest first)
    enrollment_months_count = final_df['Enrollment Month'].value_counts().sort_index(ascending=False)
    # Now format the result in the desired format
    enrollment_months_sorted = [{'month': month.strftime('%B'), 'value': count} for month, count in enrollment_months_count.items()]
    # Select the latest 4 months
    latest_4_months = enrollment_months_sorted[:4]
    # Define the inactive threshold
    inactive_threshold = pd.Timestamp.utcnow() - pd.Timedelta(days=14)

    # Filter rows where 'Last Login' is less than the inactive threshold and 'Course progress' is >= 90
    inactive_participants = final_df[
        (final_df['Last Login'] < inactive_threshold) & 
        (final_df['Course progress'] < 90)
    ].shape[0]
    total_participants = final_df.shape[0]

    count = final_df[final_df['Course progress'] >= 90].shape[0]
    count = round(count/total_participants,2)*100
    # Prepare the result as a JSON object
    result = {
        'total_participants': total_participants,
        'average_completion_rate' : count,
        "completion_percentage": round(completion_percentage,2),
        "inactive_participants": inactive_participants,
        "enrollment_months": enrollment_months_sorted,  # Updated to return counts
    }
    # Write the result to a JSON file
    with open('../data/metrics.json', 'w') as json_file:
        json.dump(result, json_file, indent=4)

    # Display the result
    print(json.dumps(result, indent=4))

In [44]:
updateMetrics(final_df, username)

{
    "total_participants": 1708,
    "average_completion_rate": 88.0,
    "completion_percentage": 90.65,
    "inactive_participants": 163,
    "enrollment_months": [
        {
            "month": "September",
            "value": 518
        },
        {
            "month": "August",
            "value": 130
        },
        {
            "month": "July",
            "value": 215
        },
        {
            "month": "June",
            "value": 200
        },
        {
            "month": "May",
            "value": 520
        },
        {
            "month": "April",
            "value": 123
        },
        {
            "month": "March",
            "value": 2
        }
    ]
}


In [35]:
def getDataRefresh():
    df = pd.read_csv('../data/scheduler.csv')
    last_row_timestamp = df['timestamp'].iloc[-1]
    date, time_part, am_pm = last_row_timestamp.rsplit(' ', 2)
    # Combine time and AM/PM
    time_str = f"{time_part} {am_pm}"
    full_datetime_str = f"{date} {time_str}"

    # Parse the full datetime
    last_datetime = datetime.strptime(full_datetime_str, "%Y-%m-%d %I:%M:%S %p")

    # Set timezone for the last data timestamp (assuming the timestamp is in EST)
    est = pytz.timezone('US/Eastern')
    last_datetime_est = est.localize(last_datetime)

    # Get the current time in UTC and convert it to EST
    current_utc_time = datetime.utcnow()
    current_est_time = pytz.utc.localize(current_utc_time).astimezone(est)

    # Calculate the time difference in minutes
    time_difference = current_est_time - last_datetime_est
    minutes_ago = int(time_difference.total_seconds() // 60)  # Convert seconds to minutes

    print(time_str)
    print(date)
    print(minutes_ago)

In [36]:
getDataRefresh()

11:35:05 PM
2024-09-28
13
